In [4]:
import spacy
from spacy import displacy
from spacy.pipeline import merge_entities
nlp = spacy.load("en_core_web_md")
# nlp.add_pipe(merge_entities)

In [2]:
def merge_two_dicts(x, y):
    """Given two dictionaries, merge them into a new dict as a shallow copy."""
    z = x.copy()
    z.update(y)
    return z

hypernym = {"POS": {"IN": ["NOUN", "PROPN"]}}
hyponym = {"POS": {"IN": ["NOUN", "PROPN"]}}
subject_deps = {"DEP": {"IN": ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]}}
object_deps = {"DEP": {"IN": ["dobj", "dative", "attr", "oprd"]}}
subject_deps = {"DEP": {"IN": ["nsubj", "nsubjpass", "csubj", "csubjpass", "agent", "expl"]}}
object_deps = {"DEP": {"IN": ["dobj", "dative", "attr", "oprd"]}}
preposition_deps = {"DEP": {"IN": ["pobj", "pcomp"]}}
hyponym = { **hyponym, **subject_deps}
hypernym = {**hypernym, **object_deps}

def join_objs(d1, lst):
    
    DEPS = []
    for entry in lst:
        DEPS += entry["DEP"]["IN"]

    return dict([("POS", d1["POS"]), ("DEP", {"IN" : d1["DEP"]["IN"] + DEPS})])

y = join_objs(hypernym, [preposition_deps, hyponym])
print(y.items())

dict_items([('POS', {'IN': ['NOUN', 'PROPN']}), ('DEP', {'IN': ['dobj', 'dative', 'attr', 'oprd', 'pobj', 'pcomp', 'nsubj', 'nsubjpass', 'csubj', 'csubjpass', 'agent', 'expl']})])


In [38]:
def doc_dep_graph(doc):
    """
    Put the graph with entity labels present (see 'tag' and 'label')
    """

    words = []  # the words of a dependency graph
    arcs = []  # the arcs of a dependency graph

    # iterate through each token in a doc
    for tok in doc:

        # if the token is not a named entity, use the named entity tag

        # create a dictionary of the words with their tags
        words.append({
            "text": tok.text,
            "tag": tok.tag_
        })

        # ignore punctuation
        if tok.dep_ not in {'nsubj', 'neg', 'conj'}:
            continue

        # if the token index is less than its head, the token is the head
        if tok.i < tok.head.i:
            arcs.append({
                "start": tok.i,
                "end": tok.head.i,
                "label": tok.dep_,
                "dir": "left"
            })

        # if the token index is greater than its head, the token head is the head
        elif tok.i > tok.head.i:
            arcs.append({
                "start": tok.head.i,
                "end": tok.i,
                "label": tok.dep_,
                "dir": "right"
            })
    return {"words": words, "arcs": arcs}

texts = [
"The Jew has never been a nomad, but always a parasite",
]

options = {"fine_grained" : True,
          "add_lemma" : True,
          "distance" : 90,
          "word_spacing" : 20,
          "collapse_punct": True}
for text in texts:
    doc = nlp(text)
    doc.user_data["title"] = "This is a title"
    displacy.render(doc, options = options)
    displacy.render(doc_dep_graph(doc), style = "dep", manual = True, options = options)

In [423]:
#     def get_modifier(self, token):
        
#         """
#         token extension to get adjectival modifiers
#         if no modifier is found, the token is returned.
#         """
        
#         for mod in token.lefts:
#             if mod.pos_ in ["ADJ"] and mod._.concept:
#                 return mod
#         return token


            token._.hasModifier = token._.get_modifier()
#             if token.pos_ in ["NOUN", "PROPN"] and token._.hasModifier.pos_ == "ADJ":
# #                 print(token._.hasModifier, 'has the fucking pos tag', token._.hasModifier.pos_)
#                 token._.hasMethod = ("hasModifier", token._.hasModifier)
#                 token._.hasModifier._.isModifier = True

'Mujahidin Brother' is an 'ingroup' because it is categorised as a 'Family Group'
'US Enemy' is an 'outgroup' because it is categorised as a 'Military Outgroup'
'Muslim friends' is an 'ingroup' because it is categorised as a 'Social Ingroup'
'United States Authorities' is 'neutral' because it is categorised as a 'Political Group'
